In [1]:
# Load data and show statistics
import pandas as pd

data = pd.read_csv("../data/Hotel_Reviews.csv")
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Additional_Number_of_Scoring,515738.0,498.081836,500.538467,1.000000,169.000000,341.000000,660.000000,2682.000000
Average_Score,515738.0,8.397487,0.548048,5.200000,8.100000,8.400000,8.800000,9.800000
Review_Total_Negative_Word_Counts,515738.0,18.539450,29.690831,0.000000,2.000000,9.000000,23.000000,408.000000
Total_Number_of_Reviews,515738.0,2743.743944,2317.464868,43.000000,1161.000000,2134.000000,3613.000000,16670.000000
Review_Total_Positive_Word_Counts,515738.0,17.776458,21.804185,0.000000,5.000000,11.000000,22.000000,395.000000
Total_Number_of_Reviews_Reviewer_Has_Given,515738.0,7.166001,11.040228,1.000000,1.000000,3.000000,8.000000,355.000000
Reviewer_Score,515738.0,8.395077,1.637856,2.500000,7.500000,8.800000,9.600000,10.000000
lat,512470.0,49.442439,3.466325,41.328376,48.214662,51.499981,51.516288,52.400181
lng,512470.0,2.823803,4.579425,-0.369758,-0.143372,0.010607,4.834443,16.429233


In [2]:
# Find the hotel with the most reviews

hotel_reviews_count = data["Hotel_Name"].value_counts()
most_reviewed_hotel = hotel_reviews_count.idxmax()
most_reviews = hotel_reviews_count.max()

print("Hotel with the most reviews:", most_reviewed_hotel)
print("Number of reviews:", most_reviews)

Hotel with the most reviews: Britannia International Hotel Canary Wharf
Number of reviews: 4789


In [3]:
data = data[data["Hotel_Name"] == most_reviewed_hotel]

In [4]:
# Step 1: Handling missing values

data.isna().sum()

Hotel_Address                                 0
Additional_Number_of_Scoring                  0
Review_Date                                   0
Average_Score                                 0
Hotel_Name                                    0
Reviewer_Nationality                          0
Negative_Review                               0
Review_Total_Negative_Word_Counts             0
Total_Number_of_Reviews                       0
Positive_Review                               0
Review_Total_Positive_Word_Counts             0
Total_Number_of_Reviews_Reviewer_Has_Given    0
Reviewer_Score                                0
Tags                                          0
days_since_review                             0
lat                                           0
lng                                           0
dtype: int64

In [5]:
# Step 2: Deduplication

print("duplicated rows:", data.duplicated().sum())
data.drop_duplicates(inplace=True)

duplicated rows: 0


In [6]:
# Step 3: Extracting the relevant columns

In [7]:
# Show hotel information

hotel_info = data[
    [
        "Hotel_Name",
        "Hotel_Address",
        "Average_Score",
        "Total_Number_of_Reviews",
        "Additional_Number_of_Scoring",
        "lat",
        "lng",
    ]
].copy()

hotel_info.drop_duplicates(inplace=True)
hotel_info

,Hotel_Name,Hotel_Address,Average_Score,Total_Number_of_Reviews,Additional_Number_of_Scoring,lat,lng
63942,Britannia International Hotel Canary Wharf,163 Marsh Wall Docklands Tower Hamlets London ...,7.1,9086,2682,51.50191,-0.023221


In [8]:
# Show review information

review_info = data[
    [
        "Review_Date",
        "Reviewer_Nationality",
        "Positive_Review",
        "Negative_Review",
        "Review_Total_Negative_Word_Counts",
        "Review_Total_Positive_Word_Counts",
        "Reviewer_Score",
        "Tags",
    ]
].copy()

review_info.to_csv("../data/Britannia.csv", index=False)